In [1]:
!pip install datasets peft

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "EMBEDDIA/sloberta"

In [4]:
from datasets import load_dataset

dataset = load_dataset("lenatr99/Slovene_SuperGLUE_RTE")

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [6]:
id2label = {0: "not_entailment", 1: "entailment"}
label2id = {"not_entailment": 0, "entailment": 1}

In [7]:
CONTEXT_COL = "premise"
HYPOTHESIS_COL = "hypothesis"
LABEL_COL = "label"

def preprocess_function(examples):
    context_inputs = [f"{CONTEXT_COL} : {x} " for x in examples[CONTEXT_COL]]
    hypothesis_inputs = [f"{HYPOTHESIS_COL} : {x} " for x in examples[HYPOTHESIS_COL]]
    inputs = [context_input + hypothesis_input for context_input, hypothesis_input in zip(context_inputs, hypothesis_inputs)]

    tokenized_examples = tokenizer(inputs, truncation=True)
    tokenized_examples[LABEL_COL] = [label2id[label] for label in examples[LABEL_COL]]

    return tokenized_examples

In [8]:
tokenized_dataset = dataset.map(preprocess_function, remove_columns=['idx', 'hypothesis', 'premise'], batched=True)

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

In [9]:
from sklearn.metrics import accuracy_score, f1_score


def compute_metrics(pred):
    label_ids = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(label_ids, preds, average="weighted")
    acc = accuracy_score(label_ids, preds)
    return {"accuracy": acc, "f1": f1}

In [10]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
from transformers import set_seed

set_seed(42)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=len(id2label), id2label=id2label, label2id=label2id
)
model.config.use_cache = False

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at EMBEDDIA/sloberta and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from peft import get_peft_model, PromptTuningConfig, TaskType

NUM_VIRTUAL_TOKENS = 12

peft_config = PromptTuningConfig(
    task_type=TaskType.SEQ_CLS,
    num_virtual_tokens=NUM_VIRTUAL_TOKENS,
    tokenizer_name_or_path=model_name,
    prompt_tuning_init="RANDOM"
)

In [13]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 601,346 || all params: 111,224,836 || trainable%: 0.5407


In [14]:
from transformers import TrainingArguments

new_model_name = "prompt_fine_tuned_rte_sloberta"

training_args = TrainingArguments(
    output_dir=new_model_name,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy='steps',
    max_steps=400,
    use_cpu=False,
    load_best_model_at_end=True
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['eval'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

max_steps is given, it will override any value given in num_train_epochs


In [16]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1
50,0.699000,0.689394,0.448276,0.389760
100,0.689500,0.683572,0.655172,0.652709
150,0.693900,0.688028,0.448276,0.389760
200,0.693700,0.690500,0.448276,0.389760
250,0.688900,0.688599,0.517241,0.487661
300,0.688000,0.688206,0.517241,0.487661
350,0.686400,0.686928,0.551724,0.532163
400,0.683500,0.686707,0.551724,0.532163


TrainOutput(global_step=400, training_loss=0.6903521728515625, metrics={'train_runtime': 38.4463, 'train_samples_per_second': 83.233, 'train_steps_per_second': 10.404, 'total_flos': 270150029177280.0, 'train_loss': 0.6903521728515625, 'epoch': 13.793103448275861})

In [17]:
trainer.evaluate()

{'eval_loss': 0.686707079410553,
 'eval_accuracy': 0.5517241379310345,
 'eval_f1': 0.5321630094043888,
 'eval_runtime': 0.1351,
 'eval_samples_per_second': 214.595,
 'eval_steps_per_second': 29.599,
 'epoch': 13.793103448275861}

In [19]:
trainer.push_to_hub(new_model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


sentencepiece.bpe.model:   0%|          | 0.00/800k [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

events.out.tfevents.1716454023.2e329e5d4728.5731.1:   0%|          | 0.00/457 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

events.out.tfevents.1716453985.2e329e5d4728.5731.0:   0%|          | 0.00/10.0k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/lenatr99/prompt_fine_tuned_rte_sloberta/commit/1796bf070cd41e4dffc7d32994999d84fefb75e5', commit_message='prompt_fine_tuned_rte_sloberta', commit_description='', oid='1796bf070cd41e4dffc7d32994999d84fefb75e5', pr_url=None, pr_revision=None, pr_num=None)

In [20]:
from transformers import pipeline

set_seed(42)

adapter_name = "lenatr99/" + new_model_name

# Example
text="premise : Potem ko je sklenila zavezništvo z glavno proti-sirijsko koalicijo, so njegovo ženo Strido na nedavnih volitvah izvolili v parlament. hypothesis : Strido so izvolili v parlament."

classifier = pipeline("sentiment-analysis", model=adapter_name)
classifier(text)

adapter_config.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at EMBEDDIA/sloberta and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'label': 'LABEL_1', 'score': 0.5385059118270874}]